In [5]:
import pandas as pd
import numpy as np
from scipy.stats import entropy, mannwhitneyu
import matplotlib.pyplot as plt
from pandas import read_csv

# Read in the feature table
feature_table_microbiota = read_csv("new_feature_table.tsv", sep="\t", skiprows=1, index_col=0)

# Transpose so rows = samples, columns = features
transposed_table = feature_table_microbiota.transpose()
feature_table_microbiota = transposed_table

#  Convert all values to numbers
column_names = list(feature_table_microbiota.columns)
for each_column in column_names:
    column_data = feature_table_microbiota[each_column]
    numeric_col = pd.to_numeric(column_data)  
    feature_table_microbiota[each_column] = numeric_col  

#  Drop rows that are completely empty
feature_table_microbiota = feature_table_microbiota.dropna(how='all')

# Load metadata and set sample name as index
metadata_35452 = read_csv("35452_mapping_file.tsv", sep="\t")
metadata_35452 = metadata_35452.set_index("sample_name")

#  Assign diet types based on run_prefix
diet_types = []
for sample in metadata_35452.index:
    prefix = metadata_35452.loc[sample, "run_prefix"]
    if prefix == "781_foliv":
        diet_types.append("Folivorous")
    elif prefix == "781_Sheets":
        diet_types.append("Non-folivorous")
    else:
        diet_types.append("Excluded")
metadata_35452["DietType"] = diet_types

#  Filter samples that are not excluded
filtered_samples = []
for samples in feature_table_microbiota.index:
    if samples in metadata_35452.index:
        if metadata_35452.loc[samples, "DietType"] != "Excluded": 
            filtered_samples.append(samples)

new_feature_table_microbiota = feature_table_microbiota.loc[filtered_samples]
new_metadata_35452 = metadata_35452.loc[filtered_samples]  

# Calculate Shannon diversity
shannon_scores = []
# Create empty lists for evenness and observed features
evenness_scores = []  
observed_features = []  

for sample in new_feature_table_microbiota.index:
    counts = new_feature_table_microbiota.loc[sample]
    total = sum(counts)
    present_features = sum(counts > 0)  
    if total == 0:
        shannon = 0
        evenness = 0  
    else:
        proportions = []
        for count in counts:
            proportion = count / total
            proportions.append(proportion)
        shannon = entropy(proportions)
        if present_features > 1:  
            evenness = shannon / np.log(present_features)  
        else:
            evenness = 0  

    shannon_scores.append(shannon)
    evenness_scores.append(evenness)  
    observed_features.append(present_features)  

# Add diversity metrics to the table
new_feature_table_microbiota["Shannon"] = shannon_scores
new_feature_table_microbiota["Evenness"] = evenness_scores  
new_feature_table_microbiota["ObservedFeatures"] = observed_features  
new_feature_table_microbiota["DietType"] = new_metadata_35452["DietType"].values

#  Separate into diet groups
folivorous = []
nonfolivorous = []

for sample in new_feature_table_microbiota.index:
    diet = new_feature_table_microbiota.loc[sample, "DietType"]
    shannon = new_feature_table_microbiota.loc[sample, "Shannon"]
    if diet == "Folivorous":
        folivorous.append(shannon)
    elif diet == "Non-folivorous":
        nonfolivorous.append(shannon)

#  Mann–Whitney U Test
stat, p = mannwhitneyu(folivorous, nonfolivorous, alternative='greater')

# Print results
print("Shannon Diversity Comparison (Mann–Whitney U Test)")
print(f"Folivorous mean:       {np.mean(folivorous):.2f}")
print(f"Non-folivorous mean:   {np.mean(nonfolivorous):.2f}")
print(f"U statistic:           {stat:.2f}")
print(f"p-value:               {p:.4f}")
if p < 0.05:
    print("Result: Folivorous group has significantly higher diversity.")
else:
    print("Result: No significant difference in diversity.")

# Plot Shannon Diversity
data = [folivorous, nonfolivorous]
labels = ["Folivorous", "Non-folivorous"]

plt.figure(figsize=(8, 6))
plt.boxplot(data, labels=labels, patch_artist=True)
plt.title("Shannon Diversity by Diet Type")
plt.ylabel("Shannon Diversity Index")
plt.grid(axis='y')
plt.savefig("Shannon_Diversity.png")

# Plot Evenness  
folivorous_evenness = new_feature_table_microbiota[new_feature_table_microbiota["DietType"] == "Folivorous"]["Evenness"]
nonfolivorous_evenness = new_feature_table_microbiota[new_feature_table_microbiota["DietType"] == "Non-folivorous"]["Evenness"]

plt.figure(figsize=(8, 6))
plt.boxplot([folivorous_evenness, nonfolivorous_evenness], labels=labels, patch_artist=True)
plt.title("Evenness by Diet Type")
plt.ylabel("Evenness")
plt.grid(axis='y')
plt.savefig("Eveness.png")

# Plot Observed Features  
folivorous_obs = new_feature_table_microbiota[new_feature_table_microbiota["DietType"] == "Folivorous"]["ObservedFeatures"]
nonfolivorous_obs = new_feature_table_microbiota[new_feature_table_microbiota["DietType"] == "Non-folivorous"]["ObservedFeatures"]

plt.figure(figsize=(8, 6))
plt.boxplot([folivorous_obs, nonfolivorous_obs], labels=labels, patch_artist=True)
plt.title("Observed Features by Diet Type")
plt.ylabel("Number of Features")
plt.grid(axis='y')
plt.savefig("Observed_Features.png")


Shannon Diversity Comparison (Mann–Whitney U Test)
Folivorous mean:       3.82
Non-folivorous mean:   3.68
U statistic:           409.00
p-value:               0.0074
Result: Folivorous group has significantly higher diversity.
